**Библиотеки**

In [120]:
import pandas as pd
import numpy as np
from utils import fetch_ohlcv_df, moex_fetch_ohlcv_df
import ccxt
from scipy.optimize import minimize

exchange = ccxt.binance({"enableRateLimit": True})

In [78]:
def Search_for_solutions(returns: pd.Series, cov: pd.DataFrame, risk: float | None = None):
    tickers = returns.index
    r = returns.values.astype(float)
    C = cov.loc[tickers, tickers].values.astype(float)
    n = len(r)

    w0 = np.ones(n) / n
    bounds = [(0.0, 1.0)] * n
    cons = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]

    # ЦЕЛЕВАЯ ФУНКЦИЯ + ОГРАНИЧЕНИЕ ПО РИСКУ
    if risk is None:
        obj = lambda w: (w.T @ C @ w)  # min variance
    else:
        obj = lambda w: -(w @ r)       # max return
        if risk != 1:
            if risk <= 0:
                raise ValueError("risk должен быть > 0, либо risk=1 для max return без ограничений")
            cons.append({"type": "ineq", "fun": lambda w: risk - np.sqrt(max(w.T @ C @ w, 0.0))})

    res = minimize(obj, w0, method="SLSQP", bounds=bounds, constraints=cons,
                   options={"maxiter": 2000, "ftol": 1e-12})

    if not res.success:
        raise RuntimeError(
            f"Заданный risk слишком мал и ограничение vol<=risk невыполнимо."
        )

    w = res.x
    w[np.abs(w) < 1e-10] = 0.0
    w = w / w.sum()

    var = float(w.T @ C @ w)
    return {
        "weights": pd.Series(w, index=tickers),
        "portfolio_risk": float(np.sqrt(max(var, 0.0))),
        "portfolio_return": float(w @ r),
    }

**Матрица доходностей**

In [36]:
ticker_list = ['BTC/USDT','ETH/USDT','TON/USDT','BNB/USDT','XRP/USDT','KAVA/USDT']

In [112]:
ticker_list = ['SBER','PLZL','MDMG','GAZP','VTBR','T','UGLD','NVTK','SELG']

In [125]:
df = moex_fetch_ohlcv_df(
    "SBER",
    "1d",
    "2025-02-02",
    "2026-02-02"
)
df

,open,high,low,close,volume
timestamp,,,,,


In [126]:
df_pct_change = dict()

for ticker in ticker_list:
    df = moex_fetch_ohlcv_df(ticker ,'1M','2025-01-01','2025-12-31') #fetch_ohlcv_df(exchange,ticker,'1M','2025-01-01','2025-12-31')
    df_pct_change[f'{ticker}'] = df['close'].pct_change().dropna()

KeyError: '1M'

In [107]:
df_pct_change = pd.DataFrame(df_pct_change)
df_pct_change

,SBER,PLZL,MDMG,GAZP,T,UGLD,VTBR,NVTK,SELG
timestamp,,,,,,,,,


**Ковариационная матрица**

In [ ]:
cov = df_pct_change.cov(ddof = 1)
cov

,BTC/USDT,ETH/USDT,TON/USDT,BNB/USDT,XRP/USDT,KAVA/USDT
BTC/USDT,0.011022,0.017130,0.008164,0.008546,0.012453,0.001631
ETH/USDT,0.017130,0.063442,0.024452,0.021143,0.029675,0.005379
TON/USDT,0.008164,0.024452,0.034054,0.012542,0.017898,0.000982
BNB/USDT,0.008546,0.021143,0.012542,0.014042,0.013158,-0.003584
XRP/USDT,0.012453,0.029675,0.017898,0.013158,0.026421,0.007618
KAVA/USDT,0.001631,0.005379,0.000982,-0.003584,0.007618,0.038023


**Риск и доходность(средняя по месяцу)**

In [42]:
std = df_pct_change.std(ddof = 1)
std

BTC/USDT     0.104987
ETH/USDT     0.251878
TON/USDT     0.184538
BNB/USDT     0.118497
XRP/USDT     0.162545
KAVA/USDT    0.194995
dtype: float64

In [43]:
returns = df_pct_change.mean()
returns

BTC/USDT    -0.008859
ETH/USDT     0.017053
TON/USDT    -0.076272
BNB/USDT     0.028888
XRP/USDT    -0.032662
KAVA/USDT   -0.123184
dtype: float64

In [ ]:
#count = len(returns)
#w = np.ones(count) / count # Вектор распределения активов в портфеле (в данном случае равномерно)
w = np.array([0,0,1,0,0,0])
portfolio_returns = np.dot(w,returns) # Средняя доходность портфеля в МЕСЯЦ

portfolio_returns

np.float64(-0.07627185305000592)

In [65]:
portfolio_var = w.T @ cov.values @ w
portfolio_risk = np.sqrt(portfolio_var)

portfolio_risk

np.float64(0.1845377705460055)

In [66]:
print(f'Риск - {portfolio_risk} \nОжидаемая доходность(за месяц) - {portfolio_returns}')

Риск - 0.1845377705460055 
Ожидаемая доходность(за месяц) - -0.07627185305000592


**ТЕСТЫ**

In [84]:
# 1. Минимальный риск
test_1 = Search_for_solutions(returns, cov)
test_1

{'weights': BTC/USDT     0.382490
 ETH/USDT     0.000000
 TON/USDT     0.000000
 BNB/USDT     0.389105
 XRP/USDT     0.000000
 KAVA/USDT    0.228405
 dtype: float64,
 'portfolio_risk': 0.08895891174347856,
 'portfolio_return': -0.02028386432256347}

In [85]:
# Максимальная доходность при заданном риске
test_2 = Search_for_solutions(returns, cov, risk=0.1)
test_2

{'weights': BTC/USDT     0.261033
 ETH/USDT     0.000000
 TON/USDT     0.000000
 BNB/USDT     0.672227
 XRP/USDT     0.000000
 KAVA/USDT    0.066740
 dtype: float64,
 'portfolio_risk': 0.1000000000000002,
 'portfolio_return': 0.008885407514364922}

In [86]:
# 3. Максимальная доходность без ограничений по риску
test_3 = Search_for_solutions(returns, cov, risk = 1)
test_3

{'weights': BTC/USDT     0.0
 ETH/USDT     0.0
 TON/USDT     0.0
 BNB/USDT     1.0
 XRP/USDT     0.0
 KAVA/USDT    0.0
 dtype: float64,
 'portfolio_risk': 0.11849716984683005,
 'portfolio_return': 0.028887941627690695}